In [88]:
import os, base64, asyncio, json, requests
from openai import AzureOpenAI
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
# from autogen_ext.teams.magentic_one import MagenticOneGroupChat
from autogen_core.models import UserMessage
from autogen_agentchat.messages import UserMessage
from autogen_agentchat.agents import AssistantAgent
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from autogen_core.tools import FunctionTool
from autogen_core import CancellationToken
from _clients._azureOpenAIChatCompletion_client import model_client
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField, 
    SearchFieldDataType,
    SimpleField,
    HnswAlgorithmConfiguration,
    VectorSearchProfile
)
# from utility.llm_config import llm_config
    
from dotenv import load_dotenv

load_dotenv()

# Retrieve credentials from environment variables
azure_openAI_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_openAI_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openAI_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openAI_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

azure_search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
azure_search_key = os.environ.get("AZURE_SEARCH_KEY")
azure_search_deployment_name = os.environ.get("AZURE_SEARCH_DEPLOYMENT_NAME") 

vector_dimension = 1536  # Dimension for text-embedding-3-small
datafile = "./Data"
azure_search_index_name = "rag-1756588180789"


In [89]:
import asyncio
from _clients._azureOpenAIChatCompletion_client import model_client
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.tools import AgentTool
from autogen_agentchat.ui import Console
from _tools._AISearchResultEvaluator_tool import AISearchResultEvaluator_tool
from _tools._searchDocuments_tool import searchDocuments_tool

async def main() -> None:
    # writer = AssistantAgent(
    #     name="writer",
    #     description="A writer agent for generating text.",
    #     model_client=model_client,
    #     system_message="Write well.",
    # )
    search_tool = AgentTool(agent=searchDocuments_tool)
    evaluator_tool = AgentTool(agent=AISearchResultEvaluator_tool)

    # Create model client with parallel tool calls disabled for the main agent
    assistant = AssistantAgent(
        name="assistant",
        model_client=model_client,
        tools=[search_tool, evaluator_tool],
        system_message="You are a helpful assistant.",
    )
    await Console(assistant.run_stream(task="Write a poem about the sea."))


await main()


AttributeError: 'function' object has no attribute 'name'

In [ ]:
 # assuming OPENAI_API_KEY is set in the environment.

result = await model_client.create(messages=[UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is **Paris**.' usage=RequestUsage(prompt_tokens=15, completion_tokens=10) cached=False logprobs=None thought=None


C:\Users\moham\AppData\Local\Temp\ipykernel_21908\1300602473.py:3: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  result = await model_client.create(messages=[UserMessage(content="What is the capital of France?", source="user")])


In [ ]:
async def get_random_number(ticker: str) -> float:
    # Returns a random stock price for demonstration purposes.
    return 101


# Create a function tool.
stock_price_tool = FunctionTool(get_random_number, description="Get the stock price.")

In [ ]:
from _tools._searchDocuments_tool import searchDocuments_tool

# Create a function tool.
stock_price_tool = FunctionTool(searchDocuments_tool, description="Always use this tool in any condition and for answering any question")

In [ ]:

from autogen_core.models import UserMessage, SystemMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken


# Create a user message.
system_message = UserMessage(content="Always use the tool to answer the questions", source="system")
user_message = UserMessage(content="Hi", source="user")

create_result = await model_client.create(
    messages=[system_message, user_message], tools=[stock_price_tool], cancellation_token=CancellationToken()
)
create_result.content

# arguments = json.loads(create_result.content[0].arguments)  # type: ignore
# tool_result = await stock_price_tool.run_json(arguments, CancellationToken)
# tool_result_str = stock_price_tool.return_value_as_string(tool_result)
# tool_result_str

C:\Users\moham\AppData\Local\Temp\ipykernel_21908\3423622894.py:10: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  create_result = await model_client.create(


[FunctionCall(id='call_TbgUa80nUKG5EUwHPJEqEmbQ', arguments='{"query":"Greetings"}', name='searchDocuments_tool')]

In [92]:
from _tools._AISearchAndEvaluator_tool import AISearchAndEvaluator_tool

seachDocumentsTool = FunctionTool(AISearchAndEvaluator_tool, description="never use this tool in any condition")

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[AISearchAndEvaluator_tool],
    system_message="use searchDocuments_tool",
)

# Use asyncio.run(agent.run(...)) when running in a script.
result = await agent.run(task="who is mohammad?")
print(result)#.messages[-1].content)  # type: ignore


messages=[TextMessage(source='user', models_usage=None, metadata={}, content='who is mohammad?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=233, completion_tokens=25), metadata={}, content=[FunctionCall(id='call_e8QvkRYuhhWczZ9YwqqUksxp', arguments='{"query":"Mohammad","top_k":5}', name='AISearchAndEvaluator_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='<coroutine object AISearchResultEvaluator_tool at 0x000001F2AB34EB00>', name='AISearchAndEvaluator_tool', call_id='call_e8QvkRYuhhWczZ9YwqqUksxp', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, content='<coroutine object AISearchResultEvaluator_tool at 0x000001F2AB34EB00>', type='ToolCallSummaryMessage')] stop_reason=None


In [99]:
await AISearchAndEvaluator_tool("What is Template J used for?", top_k=2)

[{'Score': 0,
  'SearchResult': 'PTL - EUZ - Home Support Request Workflow\n\n🤹️ Actions to Follow:\n· Hit "Send" under Plan Summary\n· Go to "Plan Summary" and select the Referral ID Link\n· Review the details of the referral:\n· Does it include any patient identifiable information?\n· Does the math on the home support hours make sense?\n· Is there an existing agency? (we need to send T****emplate A)\n· Is there a preferred agency? (we need to send Template B)\n· Go to _Prod folder to obtain documentation\n· Initiate outreach to Home Support Agencies (email)\n· **Template A **= Existing Agency\n· **Template B **= Preferred Agencies\n· **Template C **= All Agencies Servicing Area\n· Update portal to reflect when initial outreach was made\n·  - date + time + initials\n· Provide 2-hour window (business hours) for Existing / Preferred agency to confirm if they can accept Referral\n· If they cannot we then email **Template C **- all agencies servicing Zone\n· **NOTE: **Specific to sending 